In [ ]:
import os
import tweepy

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key = config.get('KEY', 'consumer_key')
consumer_secret = config.get('KEY', 'consumer_secret')
access_token = config.get('KEY', 'access_token')
access_token_secret = config.get('KEY', 'access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.get('KEY', 'json_loc')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
search_words = "Delhi Health Bulletin (from:CMODelhi)"

tweets = tweepy.Cursor(api.search,
                       q=search_words,
                       lang="en", ).items(1)

image_url = ""
# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    print("\n\n")

In [ ]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_url

response = client.text_detection(image=image)

string = ""
for text in response.text_annotations:
    string = text.description
    print(string)
    break

In [ ]:
import re
from datetime import datetime


def getDay(string):
    s = ""
    for char in string:
        if char.isdigit():
            s += char

    return int(s)


def parseString(string):
    val = dict()
    try:
        #Date
        match = (re.search("/ ", string))
        i = int(match.end())
        match = (re.search("\)", string))
        j = int(match.start())
        print(string[i:j])
        val['day'] = getDay(string[i:string.find(str(datetime.today().year)) - 1])
        val['month'] = datetime.today().month
        val['year'] = datetime.today().year

        #+ve cases
        i = string.find("Positive Cases", string.find("Positive Cases") + 1)
        match = (re.search("Tests Conducted", string))
        j = int(match.start())
        val['positive'] = string[i + len("Positive Cases") + 1: j - 1]

        #Tests
        match = (re.search("Tests Conducted", string))
        i = int(match.end())
        match = (re.search("Positivity Rate", string))
        j = int(match.start())
        val['tests'] = string[i + 1:j - 1]

        #Recovered
        match = (re.search("%", string))
        i = int(match.end())
        match = (re.search("Deaths", string))
        j = int(match.start())
        val['recovered'] = string[i + 1:j - 1]

        #Deaths
        match = (re.search("Deaths", string))
        i = int(match.end())
        match = (re.search("COVID-19 Patient Management", string))
        j = int(match.start())
        val['deaths'] = string[i + 1:j - 1]

        #Tot Vac
        match = (re.search("Beneficiaries vaccinated in last 24 hours", string))
        i = int(match.end())
        match = (re.search("Beneficiaries vaccinated 1", string))
        j = int(match.start())
        val['vaccinated'] = string[i + 1:j - 1]

        #1st dose
        match = (re.search("dose in last 24 hours", string))
        i = int(match.end())
        match = (re.search("Beneficiaries vaccinated 2", string))
        j = int(match.start())
        val['first_dose'] = string[i + 1:j - 1]

        #2nd dose
        pat = "dose in last 24 hours"
        pos = string.find(pat) + 1
        p1 = string.find(pat, pos + 1)
        i = p1 + len(pat)
        match = (re.search("Cumulative beneficiaries vaccinated so far", string))
        j = int(match.start())
        val['second_dose'] = string[i + 1:j - 1]

        #Active Cases
        match = (re.search("Active Cases", string))
        i = int(match.end())
        match = (re.search("Total Number of Containment Zones as on date", string))
        j = int(match.start())
        val['active_cases'] = string[i + 1:j - 1]

        #Containment
        match = (re.search("Total Number of Containment Zones as on date: ", string))
        i = int(match.end())
        match = (re.search("Calls received", string))
        j = int(match.start())
        val['zones'] = string[i:j - 1]
    except:
        print("/\/\/\ Error caught \/\/\/")
        return None
    print(val)
    return val

In [ ]:
val = parseString(string)

### Vaccinations

In [ ]:
import json

def parseJSON(js,date , lis , dat ):
    centres = js['centers']
    for center in centres:
        sessions = center['sessions']
        for ses in sessions:
            if (ses['date'] == date ) :
                val = dict()
                val['day'] = dat.day
                val['month'] = dat.month
                val['year'] = dat.year
                val['center_id'] = center['center_id']
                val['name'] = center['name']
                val['pincode'] = center['pincode']
                val['district_name'] = center['district_name']
                val['vaccine'] = ses['vaccine']
                val['min_age_limit'] = ses['min_age_limit']
                try :
                    if (center['fee_type'] == "Paid"):
                        val['fee'] = center['vaccine_fees'][0]['fee']
                    else :
                        val['fee'] =  0
                except:
                    val['fee'] =  0
                lis.append(val)


In [ ]:
import requests
import datetime

payload = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15'
}

i = 110001
j = 110096

lis = list()
url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode={}&date={}"
y = datetime.datetime.now()
date = y.strftime("%d-%m-%Y")

while i <= j:
    response = requests.get(url.format(i, date), headers=headers, data=payload)
    print(i)
    parseJSON(response.json(),date, lis , y)
    i += 1

In [ ]:
import json
file = "{}-centers.json".format(date)
with open(file, "w") as outfile:
    json.dump(lis, outfile)


### Uploading to Redshift

In [ ]:
import configparser
import psycopg2

config = configparser.ConfigParser()
config.read('dwh.cfg')

# Connect to DB and return the connection
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()


In [ ]:
query = """
CREATE TABLE CENTERS
(
DAY INT,
MONTH INT,
YEAR INT,
CENTER_ID INT,
NAME VARCHAR,
DISTRICT_NAME VARCHAR,
PINCODE INT,
VACCINE VARCHAR,
MIN_AGE_LIMIT INT,
FEE INT
)"""

cur.execute(query)
conn.commit()

In [ ]:
query = """
CREATE TABLE DATA
(
DAY INT,
MONTH INT,
YEAR INT,
POSITIVE INT,
TESTS INT,
RECOVERED INT,
DEATHS INT,
VACCINATED INT,
FIRST_DOSE INT,
SECOND_DOSE INT,
ACTIVE_CASES INT,
CONT_ZONES INT
)"""

cur.execute(query)
conn.commit()

In [ ]:
val['deaths'] = 332
val['recovered'] = 20160
val

In [ ]:
# Cases -> AWS RS

query = "INSERT INTO DATA " \
        "VALUES({day},{month},{year},{positive},{tests},{recovered},{deaths},{vaccinated},{first_dose},{second_dose},{active_cases},{zones})"\
        .format(**val)

cur.execute(query)
conn.commit()


In [ ]:
import boto3
# TODO
REGION = 'ap-south-1'
ACCESS_KEY_ID = config.get('KEY', 'ACCESS_ID')
SECRET_ACCESS_KEY = config.get('KEY', 'SECRET_ID')
PATH_IN_COMPUTER = file
BUCKET_NAME = 'rt-data'
KEY = file # file path in S3

s3_resource = boto3.resource(
    's3',
    region_name = REGION,
    aws_access_key_id = ACCESS_KEY_ID,
    aws_secret_access_key = SECRET_ACCESS_KEY
)

s3_resource.Bucket(BUCKET_NAME).put_object(
    Key = KEY,
    Body = open(PATH_IN_COMPUTER, 'rb')
)

In [ ]:
# Vaccine -> AWS RS

query = """
COPY centers FROM {}
CREDENTIALS 'aws_iam_role={}'
JSON {}
REGION 'ap-south-1'
COMPUPDATE OFF
"""

cur.execute(
    query.format("s3://rt-data/{}".format(file), config.get('IAM_ROLE', 'ARN'), "s3://rt-data/json-path.json")
)
conn.commit()



In [ ]:
conn.close()
s3_resource.Object(BUCKET_NAME , KEY).delete()